In [ ]:
pip install rebound

In [ ]:
import rebound
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

sim = rebound.Simulation()
sim.units = ('AU', 'days', 'Msun')

labels = ["Jupiter", "Io", "Europa", "Ganymede", "Callisto"]
sim.add(labels)


In [ ]:
os = sim.calculate_orbits()
print("n_i (in rad/days) = %6.3f, %6.3f, %6.3f" % (os[0].n,os[1].n, os[2].n))
print("P_i (in days)     = %6.3f, %6.3f, %6.3f" % (os[0].P,os[1].P, os[2].P))

In [ ]:
sim.move_to_com()
op = rebound.OrbitPlot(sim, unitlabel="[AU]", color=True, periastron=True)

In [ ]:
sim.integrator = "whfast"
sim.dt = 0.05 * os[0].P  # 5% of period
Nout = 100000            # number of points to display
tmax = 100000*365.25         # let the simulation run for X years
Nobjects = 3

In [ ]:
x = np.zeros((Nobjects,Nout))
ecc = np.zeros((Nobjects,Nout))
longitude = np.zeros((Nobjects,Nout))
varpi = np.zeros((Nobjects,Nout))

times = np.linspace(0.,tmax,Nout)
ps = sim.particles

In [ ]:
for i,time in enumerate(times):
    sim.integrate(time)
    # note we use integrate() with the default exact_finish_time=1, which changes the timestep near 
    # the outputs to match the output times we want.  This is what we want for a Fourier spectrum, 
    # but technically breaks WHFast's symplectic nature.  Not a big deal here.
    os = sim.calculate_orbits()
    for j in range(Nobjects):
        x[j][i] = ps[j+1].x 
        ecc[j][i] = os[j].e
        longitude[j][i] = os[j].l
        varpi[j][i] = os[j].Omega + os[j].omega

In [ ]:
fig = plt.figure(figsize=(12,5))
ax = plt.subplot(111)
plt.plot(times,ecc[1],label=labels[1])
plt.plot(times,ecc[2],label=labels[2])
ax.set_xlabel("Time (days)")
ax.set_ylabel("Eccentricity")
plt.legend();

In [ ]:
fig = plt.figure(figsize=(12,5))
ax = plt.subplot(111)
plt.plot(times,x[1],label=labels[1])
plt.plot(times,x[2],label=labels[2])
ax.set_xlim(0,5000)
ax.set_xlabel("Time (years)")
ax.set_ylabel("x locations (AU)")
ax.tick_params()
plt.legend();

In [ ]:
def zeroTo360(val):
    while val < 0:
        val += 2*np.pi
   # while val > 2*np.pi:
    #    val -= 2*np.pi
    return (val*180/np.pi)

def min180To180(val):
    while val < -np.pi:
        val += 2*np.pi
    while val > np.pi:
        val -= 2*np.pi
    return (val*180/np.pi)

# We can calculate theta, the resonant argument of the 3:2 neptune-pluto orbital resonance,
# which oscillates about 0 degrees:
theta = [min180To180(1.*longitude[1][i] - 1.*longitude[2][i] - varpi[0][i]) for i in range(Nout)]

# There is also a secular resonance argument, corresponding to the difference in the longitude of perihelions:
# This angle oscillates around 180 degs, with a longer period component.
theta_sec = [zeroTo360(-varpi[1][i] + varpi[0][i]) for i in range(Nout)]

fig = plt.figure(figsize=(12,5))
ax = plt.subplot(111)
ax.plot(times,theta)  
ax.plot(times,theta_sec) # secular resonance argument
ax.set_xlim([0,100*365.25])
ax.set_ylim([-180,360])
ax.set_xlabel("time (days)")
ax.set_ylabel(r"$\phi_{2:1}$")
ax.plot([0,100],[180,180],'k--')
ax.plot([0,100],[0,0],'k--')

In [ ]:
thetaL = [zeroTo360(-longitude[0][i] + 3.*longitude[1][i] - 2.*longitude[2][i]) for i in range(Nout)]

fig = plt.figure(figsize=(12,5))
ax = plt.subplot(111)

ax.plot(times,thetaL)
ax.set_ylim([0,360.])
ax.set_xlabel("time (days)")
ax.set_ylabel(r"libration argument θ3:2θ3:2\theta_{3:2}")
ax.plot([0,200],[180,180],'k--')

In [ ]:
from scipy import signal
Npts = 3000

# look for periodicities with periods logarithmically spaced between 0.01 yrs and 100 yrs
logPmin = np.log10(0.001*365.25)
logPmax = np.log10(10.*365.25)

# set up a logspaced array from 0.01 to 100 yrs
Ps = np.logspace(logPmin,logPmax,Npts)
# calculate an array of corresponding angular frequencies
ws = np.asarray([2*np.pi/P for P in Ps])

# calculate the periogram (for Io) (using ws as the values for which to compute it)
periodogram = signal.lombscargle(times,x[1],ws)

In [ ]:
fig = plt.figure(figsize=(12,5))
ax = plt.subplot(111)

# Since the computed periodogram is unnormalized, taking the value A**2*N/4, 
# we renormalize the results by applying these functions inversely to the output:
ax.set_xscale('log')
ax.set_xlim([10**logPmin,10**logPmax])
ax.set_xlabel("Period (days)")
ax.set_ylabel("Power")
ax.plot(Ps,np.sqrt(4*periodogram/Nout))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

kB = 1.38e-23   # Boltzmann constant
T = 300         # temperature in K
muB = 9.27e-24  # Bohr magneton
N = 10000       # total number of spins

B = np.linspace(0, 2, 100) # magnetic field in T
Delta = 2 * muB * B        # energy difference between up and down spins
n_down = np.exp(-Delta / (kB * T)) / (1 + np.exp(-Delta / (kB * T))) # ratio of down to total spins
M = muB * N * (2 * n_down - 1) # magnetization

plt.plot(B, M)
plt.xlabel('Magnetic field (T)')
plt.ylabel('Magnetization (Am^2)')
plt.show()